Before starting, please go to *Runtime->Change runtime type* and chose GPU as your hardware accelerator. This code will work only under this assumption.

In [ ]:
# import the relevant libraries
import numpy as np
import cv2 # openCV
import os
from google.colab.patches import cv2_imshow
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# check the opencv version
print(cv2.__version__)

4.6.0


### Cloning YOLOv4 Darknet Repository

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile

### Downloading YOLOv4 Weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
os.getcwd()

'/content'

In [ ]:
 %cd /content/drive/MyDrive/ALP_Model/darknet/custom_data

/content/drive/MyDrive/ALP_Model/darknet/custom_data


In [ ]:
%cp -r /content/drive/MyDrive/ALPR_Data/Images_LE .

In [ ]:
# copies the contenst (done by the last dot at the end of source) of first directory to the dest directory(which exists)
#https://unix.stackexchange.com/questions/180985/how-to-copy-files-from-the-folder-without-the-folder-itself
%cp -r /content/drive/MyDrive/ALPR_Data/Detect_LE/. ./Images_LE
#cp -r /home/username/A/. /usr/lib/B/

In [ ]:
# copies the first directory to current directory (as dest doesnt exist)
%cp -r /content/drive/MyDrive/ALPR_Data/Images_RP .

In [ ]:

%cp -r /content/drive/MyDrive/ALPR_Data/Detect_RP/. ./Images_RP

In [ ]:
%mv ./images/* . 
%rm -r images/

Check the image formats and convert all non jpg images to JPG format if necessary, and remove the originals

In [ ]:
%cd ./Images/

/content/darknet/custom_data/Images


### Auxiliary files creation

In [ ]:
!touch training_data_all.txt
!touch validation_data_all.txt
!touch obj_all.names
!touch obj_all.data

append class names in face_mask_classes.names

In [ ]:
!echo "License_Plate" >> obj_all.names
#!echo "face mask" >> face_mask_classes.names

configure the data file

In [ ]:
!echo "classes = 1" >> obj_all.data
!echo "train = custom_data/training_data_all.txt" >> obj_all.data
!echo "valid = custom_data/validation_data_all.txt" >> obj_all.data
!echo "names = custom_data/obj_all.names" >> obj_all.data
!echo "backup = backup_all/" >> obj_all.data

In [ ]:
!head obj_all.data

classes = 1
train = custom_data/training_data_all.txt
valid = custom_data/validation_data_all.txt
names = custom_data/obj_all.names
backup = backup/backup_all/


### Creating and populating obj.data files for AC, LE and RE

In [ ]:
!touch obj_AC.data
!touch obj_LE.data
!touch obj_RP.data

In [ ]:
!echo "classes = 1" >> obj_AC.data
!echo "train = custom_data/training_data_all_AC.txt" >> obj_AC.data
!echo "valid = custom_data/validation_data_all_AC.txt" >> obj_AC.data
!echo "names = custom_data/obj_all.names" >> obj_AC.data
!echo "backup = backup_all/" >> obj_AC.data

In [ ]:
!echo "classes = 1" >> obj_LE.data
!echo "train = custom_data/training_data_all.txt" >> obj_LE.data
!echo "valid = custom_data/validation_data_all_LE.txt" >> obj_LE.data
!echo "names = custom_data/obj_all.names" >> obj_LE.data
!echo "backup = backup_all/" >> obj_LE.data

In [ ]:
!echo "classes = 1" >> obj_RP.data
!echo "train = custom_data/training_data_all.txt" >> obj_RP.data
!echo "valid = custom_data/validation_data_all_RP.txt" >> obj_RP.data
!echo "names = custom_data/obj_all.names" >> obj_RP.data
!echo "backup = backup_all/" >> obj_RP.data

In [ ]:
%cp obj_all.data /content/drive/MyDrive/ALP_Model
%cp obj_all.names /content/drive/MyDrive/ALP_Model

### Data Folder Manipulation

In [ ]:
file_dir="/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_LE/"
files=glob.glob(file_dir+"*.txt")
#files[:5]
for f in files:
    filename = os.path.basename(f)
    os.rename(f,file_dir+"LE_"+filename)
    #print(f)
    #print(file_dir+"LE_"+filename)


In [ ]:
file_dir="/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_RP/"
files=glob.glob(file_dir+"*")
#files[:5]
for f in files:
    filename = os.path.basename(f)
    os.rename(f,file_dir+"RP_"+filename)
    #print(f)
    #print(file_dir+"RP_"+filename)

In [ ]:
def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

In [ ]:
file_dir="/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_LE/"
label_fpaths=glob.glob(file_dir+"*.txt")
#for f in label_fpaths:
    #os.remove(f)

In [ ]:
#file_dir="/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_LE/"
finallist = []
twoplates = [350,404,407,417,422,438,442,444,456,470,490,501,504,509,541,567]
for img in twoplates:
  finallist.append(file_dir+'LE_'+str(img)+'.jpg')
  finallist.append(file_dir+'LE_'+str(img)+'.txt')
  finallist.append(file_dir+'LE_'+str(img)+'_2.txt')
#finallist
for f in finallist:
    #os.remove(f)

In [ ]:
file_dir="/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_RP/"
label_fpaths=glob.glob(file_dir+"*.txt")
#files[:5]
for lbf in label_fpaths:
    #print(lbf)
    labl_filename = os.path.basename(lbf)
    #print(labl_filename)
    img_filename = labl_filename.replace(".txt",".jpg")
    #print(img_filename)
    image_path = os.path.join(file_dir,img_filename)
    img = cv2.imread(image_path)
    image_w, image_h = img.shape[1], img.shape[0]
    with open(lbf,'r+') as myfile:
      coord = [int(float(x.strip())) for x in myfile.readlines()]
      yolo_cord = [str(round(x,6)) for x in pascal_voc_to_yolo(*coord,image_w, image_h)]
      #print(lbf)
      #print(image_w, image_h )
      #print(coord)
      #print('0 '+' '.join(yolo_cord)) 
      myfile.seek(0)
      myfile.write('0 '+' '.join(yolo_cord)) 
      myfile.truncate()  
    

In [ ]:
dir ="/content/drive/MyDrive/ALPR_Data/Detect_All/"
cust_dir = "/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/"
milabl_dir = "/content/drive/MyDrive/ALPR_Data/Mislabelled/"
img_f = glob.glob(dir+'*')
mis_labelled_src =[]
mis_labelled_dest =[]
cnt = 0
for f in img_f:
  try:
    with open(f,'r') as myfile:
      coord = [int(float(x.strip())) for x in myfile.readlines()]
      
      if (coord[0]> coord[2]) or (coord[1]> coord[3]) :
      #print(f)
        cnt+=1
        mis_labelled_src.append(cust_dir+os.path.basename(f))
        mis_labelled_src.append(cust_dir+(os.path.basename(f).replace(".txt",".jpg")))
        mis_labelled_dest.append(milabl_dir+os.path.basename(f))
        mis_labelled_dest.append(milabl_dir+(os.path.basename(f).replace(".txt",".jpg")))
      #print(bb)
  except:
    print(f)
    continue
print(cnt)
print(mis_labelled_src[:5])
print(mis_labelled_dest[:5])

250
['/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/RP_117.txt', '/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/RP_117.jpg', '/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/RP_135.txt', '/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/RP_135.jpg', '/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/RP_121.txt']
['/content/drive/MyDrive/ALPR_Data/Mislabelled/RP_117.txt', '/content/drive/MyDrive/ALPR_Data/Mislabelled/RP_117.jpg', '/content/drive/MyDrive/ALPR_Data/Mislabelled/RP_135.txt', '/content/drive/MyDrive/ALPR_Data/Mislabelled/RP_135.jpg', '/content/drive/MyDrive/ALPR_Data/Mislabelled/RP_121.txt']


In [ ]:
for source,dest in zip(mis_labelled_src,mis_labelled_dest):
    os.rename(source , dest)

In [ ]:
len(os.listdir("/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All"))

3564

In [ ]:
%cp -r ./Images_LE/ ./Images_All

In [ ]:
%cp -r ./Images_AC/. ./Images_All

In [ ]:
%cp -r ./Images_RP/. ./Images_All

In [ ]:
files=glob.glob("/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/"+'*.txt')
len(files)
#len(sorted(files, key = lambda x : int(os.path.basename(x)[:-4].replace('RP_',""))))

1774

In [ ]:
%cd "/content/drive/MyDrive/ALP_Model/darknet/custom_data"

/content/drive/MyDrive/ALP_Model/darknet/custom_data


### Splitting the data into train and validation sets

In [ ]:
import random
import pandas as pd 
import os 

PATH = 'Images_All/'
list_img=[img for img in os.listdir(PATH) if img.endswith('.jpg')==True]

path_img=[PATH+img_name for img_name in list_img]
random.shuffle(path_img)
path_img

In [ ]:
path_img[:3]

['Images_All/AC_358.jpg', 'Images_All/LE_427.jpg', 'Images_All/LE_367.jpg']

In [ ]:
!pwd

/content/drive/MyDrive/ALP_Model/darknet


In [ ]:
!touch training_data_all.txt
!touch validation_data_all.txt

In [ ]:
train_path = path_img[:int(len(path_img)*0.8)]
val_path = path_img[int(len(path_img)*0.8):]
    
# relative path to the binary 
relpath = "custom_data/"

# Train file
# Open a file with access mode 'a'
with open("training_data_all.txt", "a") as file_object:
  for i in train_path:
    file_object.write(relpath+i+"\n")   
    #print(relpath+i+"\n")   

with open("validation_data_all.txt", "a") as file_object:
  for i in val_path:
    file_object.write(relpath+i+"\n")
    #print(relpath+i+"\n")

### Creating Validation Filepaths for AC, LE and RP

In [ ]:
with open("validation_data_all.txt", "r") as file_object:
  AC_f = [x for x in file_object.readlines() if 'AC_' in x]
  print(len(AC_f))
  with open("validation_data_all_AC.txt", "w") as w_f:
    w_f.write(''.join(AC_f))
  #print(''.join(AC_f))

126


In [ ]:
with open("validation_data_all.txt", "r") as file_object:
  LE_f = [x for x in file_object.readlines() if 'LE_' in x]
  print(len(LE_f))
  with open("validation_data_all_LE.txt", "w") as w_f:
    w_f.write(''.join(LE_f))
  #print(''.join(AC_f))

163


In [ ]:
with open("validation_data_all.txt", "r") as file_object:
  RP_f = [x for x in file_object.readlines() if 'RP_' in x]
  print(len(RP_f))
  with open("validation_data_all_RP.txt", "w") as w_f:
    w_f.write(''.join(RP_f))
  #print(''.join(AC_f))

66


## Backing Up

In [ ]:
%cp training_data_all.txt /content/drive/MyDrive/ALP_Model
%cp validation_data_all.txt /content/drive/MyDrive/ALP_Model

In [ ]:
%cp -r /content/drive/MyDrive/ALP_Model/darknet /content/darknet/

##  Compiling DarkNet

In [ ]:
##!chmod +x /content/drive/MyDrive/ALP_Model/darknet

In [ ]:
!make -j4

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/http_stream.cpp -o obj/http_stream.o
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/i

In [ ]:
%cd /content/drive/MyDrive/ALP_Model/darknet

/content/drive/MyDrive/ALP_Model/darknet


## Training the  Custom Model

### Bckup directory for the weights

In [ ]:
%rm -r /content/darknet/backup

In [ ]:
!ln -s /content/drive/MyDrive/ALP_Model/backup_all/ /content/darknet/

In [ ]:
final_img_dir = "/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/"
final_milablled_detect = ['RP_103','AC_312','AC_308','RP_113','RP_107','RP_89','RP_567','RP_102']

for fm in final_milablled_detect:
  ##os.rename(final_img_dir+fm+'.jpg')
  print(final_img_dir+fm+'.jpg')
  print(final_img_dir+fm+'.txt')


In [ ]:
final_img_dir = "/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_All/"
final_milablled_detect = ['RP_103','AC_312','AC_308','RP_113','RP_107','RP_89','RP_567','RP_102']

for fm in final_milablled_detect:
  #os.remove(final_img_dir+fm+'.jpg')
  #os.remove(final_img_dir+fm+'.txt')
  if os.path.exists(final_img_dir+fm+'.jpg'):
    print(final_img_dir+fm+'.jpg')
  ##print(final_img_dir+fm+'.txt')

In [ ]:
train_txt_fl= "/content/darknet/custom_data/training_data_all.txt"
final_milablled_detect = ['RP_103','AC_312','AC_308','RP_113','RP_107','RP_89','RP_567','RP_102']

with open(train_txt_fl) as ttxtf:
  for l in ttxtf:
    if (os.path.exists("/content/darknet/"+l.strip()+'.jpg') or os.path.exists("/content/darknet/"+l.strip()+'.txt')):
    #print("/content/darknet/"+l.strip()+'.jpg')
      print('y')

In [ ]:
os.path.exists("/content/drive/MyDrive/ALP_Model/darknet/custom_data/Images_LE")

True

### Model Training

Auxiliary parameters:

- **map** flag: if set, generates a raster plot/graph of the loss and mean average precision
- **dont_show** flag: if set, it prevents attempts to display the progress chart which may cause disruprions in the notebook environment

In [ ]:
!./darknet detector train custom_data/obj_all.data customization/yolov4.cfg customization/yolov4.conv.137 -map -dont_show

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 369277, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.835242), count: 2, class_loss = 0.022859, iou_loss = 9.746185, total_loss = 9.769044 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.750737), count: 2, class_loss = 0.036299, iou_loss = 3.804832, total_loss = 3.841130 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000066, iou_loss = 0.000000, total_loss = 0.000066 
 total_bbox = 369281, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.646127), count: 6, class_loss = 2.103449, iou_loss = 32.834026, total_loss = 34.937477 


In [ ]:
#!./darknet detector m!./darknet detector map custom_data/obj.data customization/yolov4.cfg backup/yolov4_best.weights -thresh 0.3
#ap custom_data/obj.data customization/yolov4.cfg backup/yolov4_best.weights -thresh 0.3

### Model Training Continued from First interruption point

In [ ]:
!./darknet detector train custom_data/obj_all.data customization/yolov4.cfg /content/drive/MyDrive/ALP_Model/bacukp_all_996_Iter/yolov4_last.weights -map -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 108868, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, class_loss = 0.000834, iou_loss = 0.000000, total_loss = 0.000834 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.794697), count: 3, class_loss = 0.008529, iou_loss = 9.425059, total_loss = 9.433588 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.826676), count: 1, class_loss = 0.007580, iou_loss = 2.203751, total_loss = 2.211331 
 total_bbox = 108872, rewritten_bbox = 0.000000 % 

 (next mAP calculation at 1200 iterations) 
 Last accuracy mAP@0.50 = 98.80 %, best = 99.15 % ]2;1181/1200: loss=0.5 map=0.99 best=0.99 hours left=0.3
 1181: 0.480152, 0.574791 avg loss, 0.000130 rate, 12.377011 seconds, 75584 images, 0.295501 hours left
Loaded: 0.000059 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.0

### Model Training Continued from second interruption point

In [ ]:
!./darknet detector train custom_data/obj_all.data customization/yolov4.cfg /content/drive/MyDrive/ALP_Model/backup_all/yolov4_last.weights -map -dont_show

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.757720), count: 3, class_loss = 1.325529, iou_loss = 40.814133, total_loss = 42.139664 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.839415), count: 4, class_loss = 0.541438, iou_loss = 17.782345, total_loss = 18.323782 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.901899), count: 1, class_loss = 0.052938, iou_loss = 3.802174, total_loss = 3.855112 
 total_bbox = 186661, rewritten_bbox = 0.000000 % 

 (next mAP calculation at 2000 iterations) 
 Last accuracy mAP@0.50 = 99.69 %, best = 99.69 % ]2;1981/2000: loss=0.6 map=1.00 best=1.00 hours left=0.3
 1981: 0.630464, 0.542219 avg loss, 0.000130 rate, 12.898816 seconds, 126784 images, 0.324039 hours left
Loaded: 0.000046 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, c

### Evaluating the Final model

In [ ]:
!./darknet detector map custom_data/obj_RP.data customization/yolov4.cfg /content/drive/MyDrive/ALP_Model/backup_all/yolov4_best.weights -thresh 0.8
#-iou_thresh 0.7
#-thresh 0.8

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x

It timed out after  9h 17m 45s with approximately 95% of the iterations completed. We used the best weights obtained 